## Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import scipy.signal as sp

In [2]:
columns = ['sample_index', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'A1', 'A2', 'A3', 'timestamp']

In [3]:
sean = pd.read_table('recorded_data/Seantry4_GoodandBad/OpenBCI-RAW-Seantry4_GoodandBad.txt', delimiter=',', names=columns)

In [4]:
sean_time = np.loadtxt('recorded_data/Seantry4_GoodandBad/sean_good_bad_23.57_time')

In [5]:
sean = sean.dropna()

In [6]:
sec = []
yr = "2018-04-07"
for i in sean.timestamp:
    str_fixed = yr + i
    sec.append(datetime.datetime.timestamp(datetime.datetime.strptime(str_fixed, "%Y-%m-%d %H:%M:%S.%f")))

In [7]:
sec = pd.Series(sec, name = 'sec')

In [8]:
data = pd.concat([sean, sec], axis = 1)

In [9]:
start = sec[0]

fixed_time = [start]
for i in range(754100):
    fixed_time.append(fixed_time[-1]+(1/250))
    
fixed_time = pd.Series(fixed_time, name = 'fixed_time')
fixed_time = fixed_time[:data.sec.shape[0]]
fixed_time.shape

(748026,)

In [10]:
data = pd.concat([sean, fixed_time], axis = 1)

In [11]:
exp_beg = data[abs(fixed_time - sean_time[0]) <= 2*1e-3].index[0]

In [12]:
nonfiltered_data = data[exp_beg:].drop(['timestamp', 'A1', 'A2', 'A3'], axis = 1).reset_index(drop = True)

In [13]:
no_white = nonfiltered_data[2560:].reset_index(drop = True)

In [14]:
# load pictures:
pict = np.loadtxt('recorded_data/Seantry4_GoodandBad/sean_good_bad_23.57_pictures_only', dtype='<U9')

In [15]:
pict.shape

(246,)

In [16]:
ans = []
for i in pict:
    if i[0] == 'P':
        ans.append(1) #1 = positive
    else:
        ans.append(0) #0 -negative
    ans.append(3) #3 - black
ans = np.array(ans)

In [17]:
ans.shape[0]

492

In [18]:
no_white.shape

(742253, 10)

In [19]:
#no_white.head()

In [20]:
lbl_data = []

for i in range(ans.shape[0]):
    if ans[i] != 3:
        lbl_data.append(no_white[i:i+8*250])
    if ans[i] == 3:
        lbl_data.append(no_white[i:i+4*250])

In [21]:
#lbl_data[0].head()

In [22]:
dataset = []

for i in lbl_data:
    feature = []
    for j in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']:
        ch = i[j]
        f, Pxx_spec = sp.welch(ch, 250, nperseg = 250)
        alpha_ch = np.mean(Pxx_spec[(f>=8) & (f<=13)])
        beta_ch = np.mean(Pxx_spec[(f>13) & (f<=30)])
        theta_ch = np.mean(Pxx_spec[(f>=4) & (f<=7)])
        gamma_ch = np.mean(Pxx_spec[(f>=30) & (f<=50)])
        feature.append(alpha_ch)
        feature.append(beta_ch)
        feature.append(theta_ch)
        feature.append(gamma_ch)    
    dataset.append(feature)

In [23]:
cols = []
for j in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']:
    col = [i+'_' +j for i in ['alpha', 'beta', 'theta', 'gamma']]
    cols +=col

In [24]:
processed_data = pd.DataFrame(dataset, columns=cols)

## Training the model

In [25]:
from sklearn.svm import SVC

In [26]:
#clf = SVC()#class_weight = 'balanced')
#clf.fit(processed_data[:400], ans[:400]) 

In [27]:
from sklearn.metrics import accuracy_score
#ans_pred= clf.predict(processed_data[400:])
#accuracy_score(ans[400:], ans_pred)

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000, max_depth=100000, random_state = 0,class_weight = 'balanced')
rf.fit(processed_data[:400], ans[:400]) 

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=100000, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [29]:
ans_pred= rf.predict(processed_data[:400])

print('Accuracy on 18% of data:', accuracy_score(ans[:400], ans_pred))

Accuracy on 18% of data: 1.0


In [30]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, processed_data, ans, cv=4)
print('Cross-validation on 6 folds score:',np.mean(scores))

Cross-validation on 6 folds score: 0.643010662401706


In [31]:
#DEMO OPENBCI

#import time
#def follow(thefile):
    #thefile.seek(0,2)
#    while True:
#        line = thefile.readline()
#        if not line:
#            time.sleep(0.1)
#            continue
#        yield line

#if __name__ == '__main__':
#logfile = open("sean_EEG","r")
#loglines = follow(logfile)
#current_data = []
#for line in loglines:
#    if (line[0:1] != '\n') and (line[0:1] != '--') and (line[0:1] != 'ID') and (line.count('.') != 3):
#        my_float_list = [float(x) for x in line[:-1].split(',')]
#        current_data.append(my_float_list)
#    if len(current_data) == 1:
#        print(line)
    #pass
    #print(line)

In [32]:
#current_data

In [33]:
no_white.head()

,sample_index,C1,C2,C3,C4,C5,C6,C7,C8,fixed_time
0,254,76513.39,131766.45,33177.09,131464.44,83188.27,183180.31,154597.98,174303.56,1.523167e+09
1,255,76456.30,131766.44,33188.96,131413.30,83199.52,183180.78,154584.03,174310.98,1.523167e+09
2,0,76337.04,131742.30,33211.14,131348.88,83211.81,183176.25,154591.70,174313.78,1.523167e+09
3,1,76386.05,131744.16,33210.45,131382.13,83209.82,183176.86,154601.16,174319.33,1.523167e+09
4,2,76516.57,131771.44,33195.38,131458.20,83195.34,183176.61,154596.80,174316.88,1.523167e+09


In [34]:
rf.predict(processed_data[1:2])[0]

3

In [35]:
def feat_extract(i):
    feature = []
    for j in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']:
        ch = i[j]
        f, Pxx_spec = sp.welch(ch, 250, nperseg = 250)
        alpha_ch = np.mean(Pxx_spec[(f>=8) & (f<=13)])
        beta_ch = np.mean(Pxx_spec[(f>13) & (f<=30)])
        theta_ch = np.mean(Pxx_spec[(f>=4) & (f<=7)])
        gamma_ch = np.mean(Pxx_spec[(f>=30) & (f<=50)])
        feature.append(alpha_ch)
        feature.append(beta_ch)
        feature.append(theta_ch)
        feature.append(gamma_ch)  
    return np.array(feature)

In [36]:
def show_emotion(prediction):
    if prediction[0] == 1: #positive
        img = cv.imread('experiment/emotions/happy.jpg',-1)
    if prediction[0] == 0: #negative
        img = cv.imread('experiment/emotions/sad.png',-1)
    if prediction[0] == 3: #black
        img = cv.imread('experiment/emotions/Neutral.png',-1)
    return img

In [37]:
pict.shape

(246,)

In [44]:
import cv2 as cv
import os
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

cv.namedWindow("picture", cv.WINDOW_NORMAL)
cv.namedWindow("emotion", cv.WINDOW_NORMAL)
#cv.imshow('window',white_image)
#key = cv.waitKey(10000)
black_image = np.zeros((600, 600, 3), np.uint8)
it = 0
present_data = lbl_data[82*5:]
for name in pict[41*5:]:
    #cv.setWindowProperty("window",cv.WND_PROP_FULLSCREEN,cv.WINDOW_FULLSCREEN)
    img = cv.imread('experiment/pictures/'+name,-1)
    current = present_data[it]
    
    to_predict = [feat_extract(current)]
    prediction = rf.predict(to_predict)
    em = show_emotion(prediction)
    
    cv.imshow('picture', img)
    cv.imshow('emotion', em)
    key = cv.waitKey(3000)
    it += 1
    current = present_data[it]
    
    to_predict = [feat_extract(current)]
    prediction = rf.predict(to_predict)
    em = show_emotion(prediction)
    
    cv.imshow('picture', black_image)
    cv.imshow('emotion', em)
    key = cv.waitKey(1500)
    it += 1
    if key == 27: # exit on ESC
        break
cv.destroyAllWindows()